<a href="https://colab.research.google.com/github/jjpeh/W207/blob/master/W207_Home_Price_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>